In [15]:
! git clone https://github.com/wchill/HMP_Dataset.git

Cloning into 'HMP_Dataset'...
fatal: unable to access 'https://github.com/wchill/HMP_Dataset.git/': Failed to connect to github.com port 443: Connection timed out


In [2]:
from pyspark.sql.types import StructType, StructField, IntegerType 

schema = StructType([StructField('x', IntegerType(), True),
                     StructField('y', IntegerType(), True),
                     StructField('z', IntegerType(), True)])

In [3]:
import os
from pyspark.sql.functions import lit
file_list = os.listdir('HMP_Dataset') #creating a list from the files.
file_list_filtered = {s for s in file_list if '_' in s} # selecting files containning only _ meaning the files with original data and ignoring the rest.

df = None # init dataframe

for category in file_list_filtered:
    data_files = os.listdir('HMP_Dataset/'+category)
    
    for data_file in data_files:
        print(data_file)
        
        temp_df = spark.read.option('header', 'false').option('delimiter', ' ').csv('HMP_Dataset/'+category+'/'+data_file, schema = schema)
        temp_df = temp_df.withColumn('class', lit(category))
        temp_df = temp_df.withColumn('source', lit(data_file))
        
        if df is None :
            df = temp_df
        else: 
            df = df.union(temp_df)

FileNotFoundError: [Errno 2] No such file or directory: 'HMP_Dataset'

In [10]:
from pyspark.ml.feature import StringIndexer 
indexer = StringIndexer(inputCol = 'class', outputCol = 'classIndex')
from pyspark.ml.feature import OneHotEncoder

encoder = OneHotEncoder(inputCol = 'classIndex', outputCol = 'categoryVec')

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

vectorAssembler = VectorAssembler(inputCols = ['x', 'y', 'z'], outputCol = 'features')

from pyspark.ml.feature import Normalizer
normalizer = Normalizer(inputCol = 'features', outputCol = 'features_norm')

In [11]:
df_two_data = spark.sql("""
select * from df where class in ('Use_telephone', 'Standup_chair')
""")

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



AnalysisException: 'Table or view not found: df; line 2 pos 14'

In [12]:
splits = df_two_data.randomSplit([0.8, 0.2])
df_train = splits[0]
df_test = splits[1]

NameError: name 'df_two_data' is not defined

In [13]:
from pyspark.ml.classification import LinearSVC

lsvc = LinearSVC(maxIter = 18, regParam = 0.1)

In [14]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages = [indexer, encoder, vectorAssembler, normalizer, lsvc ])

In [ ]:
model = pipeline.fit(df_train)
prediction = model.predict(df_train) 
from pyspark.ml.evaluator import BinaryClassificationEvaluator
eval = BinaryClassificationEvaluator